In [ ]:
# This example needs additional packages:
# !pip install matplotlib umap-learn

# for development:
# !pip install -e ".[dev]"

In [ ]:
# %load_ext autoreload
# %autoreload 2
# %env ANYWIDGET_HMR=1
from glob import glob

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from umap import UMAP

from dimbridge import Dimbridge

plt.style.use("ggplot")
plt.style.use("seaborn-v0_8-colorblind")

## Minimal example: Synthetic Data

In [ ]:
# n = int(2e4)

# ## data
# R = 2
# P = 3
# eps = 0.5
# u = np.random.rand(n) * np.pi * 2
# v = np.random.rand(n) * np.pi * 2

# x = R * (np.cos(u / 2) * np.cos(v) - np.sin(u / 2) * np.sin(2 * v))
# y = R * (np.sin(u / 2) * np.cos(v) + np.cos(u / 2) * np.sin(2 * v))
# z = P * np.cos(u) * (1 + eps * np.sin(v))
# w = P * np.sin(u) * (1 + eps * np.sin(v))

In [ ]:
# ## construct pandas dataframe and compute UMAP
# df = pd.DataFrame(dict(x1=x, x2=y, x3=z, x4=w))
# # xy = UMAP(n_neighbors=50, min_dist=0.3).fit_transform(df.to_numpy())
# xy = np.c_[u, v]

# ## validate UMAP
# plt.figure(figsize=[3, 3])
# plt.scatter(xy[:, 0], xy[:, 1], s=1)
# plt.axis("equal")
# plt.show()

In [ ]:
# # for dev testing:
# from importlib import reload
# import dimbridge

# reload(dimbridge)
# from dimbridge import Dimbridge


# dimbridge = Dimbridge(
#     data=df,
#     x=xy[:, 0],
#     y=xy[:, 1],
#     dataset_name="",
#     s=4,  # projection plot mark size
#     splom_s=1,
#     predicate_mode="data extent",  # "data extent", "predicate regression"
#     brush_mode="curve",  # 'single', "contrastive", "curve",
# )
# dimbridge

## Animal Data in the paper

- Download the data and place it under /dataset
- run python app.py

In [ ]:
# dataset_name = "gait2"
# df = pd.read_csv(f"./datasets/{dataset_name}/{dataset_name}.csv")
# df = df.drop(columns=["x", "y"])
# for col in df.columns:
#     if df[col].dtype == "int64":
#         df[col] = df[col].astype("int32")
# # xy = np.c_[df["leg1.joint1.angle"].to_numpy(), df["leg1.joint2.angle"].to_numpy()]
# numeric_columns = [col for col in df.columns if df[col].dtype != "int32"]
# xy = UMAP(n_neighbors=30, min_dist=0.2).fit_transform(df[numeric_columns].to_numpy())


dataset_name = "animals5_remote"
df = pd.read_csv(glob(f"./datasets/{dataset_name}/*.csv")[0])
image_urls = df["image_url"].to_list()
xy = df[["x", "y"]].to_numpy()
df = df.drop(columns=["x", "y", "image_filename", "image_url"])
xy = UMAP(n_neighbors=50, min_dist=0.8).fit_transform(df.to_numpy())

In [ ]:
## validate UMAP
plt.figure(figsize=[3, 3])
plt.scatter(xy[:, 0], xy[:, 1], s=3)
plt.axis("equal")
plt.show()

In [ ]:
# for dev testing:
from importlib import reload
import dimbridge

reload(dimbridge)
from dimbridge import Dimbridge

dimbridge = Dimbridge(
    data=df,
    dataset_name=dataset_name,
    predicate_host="http://localhost:9001",
    image_urls=image_urls,
    x=xy[:, 0],
    y=xy[:, 1],
    # initial color
    # c=np.zeros(xy.shape[0]).astype(np.int32),
    # cmap="set10",
    s=4,  # projection plot mark size
    splom_s=2,  # SPLOM plot mark size
    predicate_mode="predicate regression",  # "data extent", "predicate regression"
    splom_attributes=df.columns[:5].tolist(),
    brush_mode="curve",  # 'single', "contrastive", "curve",
)

dimbridge

In [ ]:
# todo export svg / pdf
# todo how-to, readme
#